In [67]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd
from datetime import date, datetime
import holidays

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Barra de progreso de un proceso
# ------------------------------------------------------------------------------
from tqdm import tqdm

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

# Poner descansos en código.
# ------------------------------------------------------------------------------
import time

# Librería para crear archivos pickle.
# ------------------------------------------------------------------------------
import pickle
import os

In [40]:
#Abrimos el pickle creado y aplicamos el modelo almacenado en él.
with open('data/modelo_rf_totales.pkl', 'rb') as f:
    forest=pickle.load(f)

In [41]:
#Comprobamos que el modelo se ha cargado adecuadamente.
forest

RandomForestRegressor(max_depth=15, max_features=3, min_samples_leaf=2,
                      min_samples_split=1)

## Prueba del modelo

In [21]:
prediccion={'clima': 2.0,
 'temperatura': 14.9,
 'humedad': 70.0,
 'viento': 16.7,
 'total': 801.0,
 'festividad_1': 0.0,
 'mes_bueno': 1.0,
 'dia_semana_nuevo': 1.0,
 'no_laboral_nuevo': 1.0,
 'estacion_map': 0.0,
 'año_map_23': 0.0,
 'año_map_18': 1.0,
 'año_map_19': 0.0}

In [23]:
df_pred=pd.DataFrame(prediccion,index=[0])
df_pred

,clima,temperatura,humedad,viento,total,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,estacion_map,año_map_23,año_map_18,año_map_19
0,2.0,14.9,70.0,16.7,801.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0


In [35]:
x_pred=df_pred.drop(['total'],axis=1)

In [42]:
forest.predict(x_pred)

array([1292.62646757])

## Prueba de la API

In [48]:
import pandas as pd
import numpy as np
import requests

from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all"

import ast
pd.options.display.max_columns = None

In [56]:
def obtener_clima(dicc, producto):
    #Iteramos por el diccionario con las localizaciones
    lista_dataframes = []
    for key, value in dicc.items():
        lat = value[0]
        lon = value[1]
    #Llamamos a la API
        url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'
        response = requests.get(url=url)
        codigo_estado = response.status_code
        razon_estado = response.reason
    #Ponemos esto para comprobar si todo está correcto.    
        if codigo_estado == 200:
            print('La peticion de se ha realizado correctamente, código de estado:',codigo_estado,'razón:',razon_estado)
        elif codigo_estado == 402:
            print('No se ha podido autorizar al usuario, código de estado:', codigo_estado,'razón:',razon_estado)
        elif codigo_estado == 404:
            print('Algo ha salido mal, el recurso no se ha encontrado,código de estado:', codigo_estado,'razón:',razon_estado)
        else:
            print('Ha ocurrido algo inesperado, código de estado:', codigo_estado,'razón:',razon_estado)
    #Convertimos los resultados en un dataframe:
        df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
    #Creamos columnas con el nombre del pais, la latitud y la longitud para poder hacer la union.
        df["place"] = key
        df["latitud"] = lat
        df["longitud"] = lon
    #Apendeamos el dataframe en una lista que hemos creado
        lista_dataframes.append(df)
    
    #Unimos los dataframes en uno
    df_lugares = pd.concat(lista_dataframes, axis= 0, ignore_index= True)
    return df_lugares

In [57]:
#Creamos un diccionario con la localización de Washington D.C., con la posibilidad de añadir más localizaciones.
dict_lugares={'Washington':[38.897588, -77.036496]}
#Llamamos a la función.
df_meteo = obtener_clima(dict_lugares,'meteo')
df_rh=df_meteo['rh_profile'].apply(pd.Series)
for i in range(len(df_rh.columns)):
    nombre = "rh_" + str(df_rh[i].apply(pd.Series)["layer"][0])
    valores = list(df_rh[i].apply(pd.Series)["rh"])
    df_meteo.insert(i, nombre, valores)
df_wind = df_meteo["wind_profile"].apply(pd.Series)
for i in range(len(df_wind.columns)):
    nombre1 = "wind_direction" + str(df_wind[i].apply(pd.Series)["layer"][0])
    valores1 = list(df_wind[i].apply(pd.Series)["direction"])
    df_meteo.insert(i, nombre1, valores1)
for i in range(len(df_wind.columns)):
    nombre2= "wind_speed" + str(df_wind[i].apply(pd.Series)["layer"][0])
    valores2 = list(df_wind[i].apply(pd.Series)["speed"])
    df_meteo.insert(i, nombre2, valores2)
df_meteo.drop(["rh_profile", "wind_profile"], axis= 1, inplace= True)
df_meteo_media = df_meteo.groupby("place").mean().reset_index()
df_meteo_media.head(3)

La peticion de se ha realizado correctamente, código de estado: 200 razón: OK


/tmp/ipykernel_9280/3603579608.py:20: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_meteo_media = df_meteo.groupby("place").mean().reset_index()


,place,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,wind_speed450mb,wind_speed400mb,wind_speed350mb,wind_speed300mb,wind_speed250mb,wind_speed200mb,wind_direction950mb,wind_direction900mb,wind_direction850mb,wind_direction800mb,wind_direction750mb,wind_direction700mb,wind_direction650mb,wind_direction600mb,wind_direction550mb,wind_direction500mb,wind_direction450mb,wind_direction400mb,wind_direction350mb,wind_direction300mb,wind_direction250mb,wind_direction200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.speed,latitud,longitud
0,Washington,2.421875,2.78125,3.015625,3.140625,3.34375,3.453125,3.703125,4.015625,4.15625,4.265625,4.359375,4.40625,4.71875,4.84375,4.59375,4.28125,92.109375,120.546875,147.578125,161.171875,165.625,168.59375,175.0,176.953125,176.25,186.25,185.859375,194.921875,201.171875,203.125,207.578125,217.8125,10.78125,8.109375,5.0625,5.609375,5.140625,4.15625,3.4375,2.3125,1.3125,1.609375,2.234375,4.21875,6.390625,9.5,6.171875,1.203125,97.5,5.640625,-9999.0,-9999.0,-9999.0,11.515625,8.875,10.625,1030.984375,1.5,0.0,2.4375,38.897588,-77.036496


In [58]:
prediccion

{'clima': 2.0,
 'temperatura': 14.9,
 'humedad': 70.0,
 'viento': 16.7,
 'total': 801.0,
 'festividad_1': 0.0,
 'mes_bueno': 1.0,
 'dia_semana_nuevo': 1.0,
 'no_laboral_nuevo': 1.0,
 'estacion_map': 0.0,
 'año_map_23': 0.0,
 'año_map_18': 1.0,
 'año_map_19': 0.0}

In [103]:
#Creamos un diccionario con la localización de Washington D.C., con la posibilidad de añadir más localizaciones.
dict_lugares={'Washington':[38.897588, -77.036496]}
#Llamamos a la función para obtener la predicción de los próximos 7 días.
df_meteo = obtener_clima(dict_lugares,'civillight')
df_meteo

La peticion de se ha realizado correctamente, código de estado: 200 razón: OK


,date,weather,wind10m_max,temp2m.max,temp2m.min,place,latitud,longitud
0,20230212,pcloudy,2,11,10,Washington,38.897588,-77.036496
1,20230213,clear,3,11,9,Washington,38.897588,-77.036496
2,20230214,cloudy,3,12,11,Washington,38.897588,-77.036496
3,20230215,cloudy,3,12,12,Washington,38.897588,-77.036496
4,20230216,mcloudy,3,12,11,Washington,38.897588,-77.036496
5,20230217,pcloudy,3,12,11,Washington,38.897588,-77.036496
6,20230218,cloudy,3,12,11,Washington,38.897588,-77.036496


In [82]:
df_prueba1=df_meteo.copy()
df_prueba1.columns

Index(['date', 'weather', 'wind10m_max', 'temp2m.max', 'temp2m.min', 'place',
       'latitud', 'longitud'],
      dtype='object')

In [83]:
#Obtenemos la fecha de la que estamos haciendo la predicción.
df_prueba1['fecha_nueva'] = pd.to_datetime(df_prueba1["date"], format="%Y%m%d")

In [84]:
#Obtenemos si ese día es festivo o no.
df_prueba1['festividad'] = pd.Series(df_prueba1.fecha_nueva).apply(lambda x: holidays.CountryHoliday('US').get(x)).values
df_prueba1['festividad_1'] = df_prueba1['festividad'].astype('bool').astype('int')

/home/solkiria/anaconda3/lib/python3.9/site-packages/holidays/utils.py:266: DeprecationWarning: CountryHoliday is deprecated, use country_holidays instead.
  warnings.warn(


In [85]:
#Obtenemos la columna del año
df_prueba1['año_bueno'] = pd.DatetimeIndex(df_prueba1['fecha_nueva']).year

In [72]:
#Creamos una función para obtener la estación a la que pertenece esa fecha.
def sacar_estacion(fecha):
    if fecha.month == 2 or fecha.month == 1:
        return "invierno"
    elif fecha.month == 3 and fecha.day <= 20:
        return "invierno"
    elif fecha.month == 12 and fecha.day >20:
        return "invierno"
    elif fecha.month == 4 or fecha.month == 5:
        return "primavera"
    elif fecha.month == 3 and fecha.day >20:
        return "primavera"
    elif fecha.month == 6 and fecha.day <=20:
        return "primavera"
    elif fecha.month == 7 or fecha.month == 8:
        return "verano"
    elif fecha.month == 6 and fecha.day > 20:
        return "verano"
    elif fecha.month == 9 and fecha.day < 23:
        return "verano"
    elif fecha.month == 10 or fecha.month == 11:
        return "otoño"
    elif fecha.month == 9 and fecha.day > 22:
        return "otoño"
    elif fecha.month == 12 and fecha.day <= 20:
        return "otoño"

In [86]:
#Obtenemos la estación
df_prueba1["estacion_correcta"] = df_prueba1["fecha_nueva"].apply(sacar_estacion)

In [87]:
#Obtenemos el mes
df_prueba1['mes_bueno'] = pd.DatetimeIndex(df_prueba1['fecha_nueva']).month

In [88]:
#Obtenemos el día de la semana
df_prueba1['dia_semana_nuevo']=df_prueba1['fecha_nueva'].dt.weekday

In [79]:
#Creamos una funcion para crear una columna con los dias laborales.
def sacar_laboral(df):
    if (df['dia_semana_nuevo']<=4) and (df['festividad_1']== 0):
        return 1
    else:
        return 0

In [89]:
#Obtenemos los días laborales
df_prueba1['no_laboral_nuevo']=df_prueba1.apply(sacar_laboral,axis=1)

In [90]:
df_prueba1

,date,weather,wind10m_max,temp2m.max,temp2m.min,place,latitud,longitud,fecha_nueva,festividad,festividad_1,año_bueno,estacion_correcta,mes_bueno,dia_semana_nuevo,no_laboral_nuevo
0,20230212,pcloudy,2,11,10,Washington,38.897588,-77.036496,2023-02-12,None,0,2023,invierno,2,6,0
1,20230213,clear,3,11,9,Washington,38.897588,-77.036496,2023-02-13,None,0,2023,invierno,2,0,1
2,20230214,cloudy,3,12,11,Washington,38.897588,-77.036496,2023-02-14,None,0,2023,invierno,2,1,1
3,20230215,cloudy,3,12,12,Washington,38.897588,-77.036496,2023-02-15,None,0,2023,invierno,2,2,1
4,20230216,mcloudy,3,12,11,Washington,38.897588,-77.036496,2023-02-16,None,0,2023,invierno,2,3,1
5,20230217,pcloudy,3,12,11,Washington,38.897588,-77.036496,2023-02-17,None,0,2023,invierno,2,4,1
6,20230218,cloudy,3,12,11,Washington,38.897588,-77.036496,2023-02-18,None,0,2023,invierno,2,5,0


Tiempos meterológicos del modelo:
- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

Posibles tiempos meteorologicos de la API clasificados según nuestro modelo:
- 1: clear,pcloudy,mcloudy,cloudy
- 2: humid
- 3: lightrain, oshower, ishower, lightsnow,ts
- 4: rain, rainsnow, tsrain, snow

In [117]:
#Obtenemos el clima:
condiciones=[df_prueba1['weather'].isin(['clear','pcloudy','mcloudy','cloudy']),
            df_prueba1['weather'].isin(['humid']),
            df_prueba1['weather'].isin(['lightrain', 'oshower', 'ishower', 'lightsnow','ts']),
            df_prueba1['weather'].isin(['rain', 'rainsnow', 'tsrain', 'snow'])]
opciones=[1,2,3,4]
df_prueba1['clima'] =np.select(condiciones,opciones)

In [123]:
#Obtenemos la temperatura
def obtener_temp(df):
    df['temperatura']=(df['temp2m.min'] + df['temp2m.max'])/2

In [124]:
obtener_temp(df_prueba1)

In [126]:
#Obtenemos viento
df_prueba1['viento']=df_prueba1['wind10m_max']

In [127]:
#Obtenemos humedad
df_prueba1['humedad']=65

In [ ]:
#Obtenemos la sensacion termica
df_prueba1['sens_termica'] = df_prueba1['temperatura']

In [125]:
x_pred

,clima,temperatura,humedad,viento,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,estacion_map,año_map_23,año_map_18,año_map_19
0,2.0,14.9,70.0,16.7,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0


In [129]:
x_pred.columns

Index(['clima', 'temperatura', 'humedad', 'viento', 'festividad_1',
       'mes_bueno', 'dia_semana_nuevo', 'no_laboral_nuevo', 'estacion_map',
       'año_map_23', 'año_map_18', 'año_map_19'],
      dtype='object')

In [128]:
df_prueba1

,date,weather,wind10m_max,temp2m.max,temp2m.min,place,latitud,longitud,fecha_nueva,festividad,festividad_1,año_bueno,estacion_correcta,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,clima,temperatura,viento,humedad
0,20230212,pcloudy,2,11,10,Washington,38.897588,-77.036496,2023-02-12,None,0,2023,invierno,2,6,0,1,10.5,2,65
1,20230213,clear,3,11,9,Washington,38.897588,-77.036496,2023-02-13,None,0,2023,invierno,2,0,1,1,10.0,3,65
2,20230214,cloudy,3,12,11,Washington,38.897588,-77.036496,2023-02-14,None,0,2023,invierno,2,1,1,1,11.5,3,65
3,20230215,cloudy,3,12,12,Washington,38.897588,-77.036496,2023-02-15,None,0,2023,invierno,2,2,1,1,12.0,3,65
4,20230216,mcloudy,3,12,11,Washington,38.897588,-77.036496,2023-02-16,None,0,2023,invierno,2,3,1,1,11.5,3,65
5,20230217,pcloudy,3,12,11,Washington,38.897588,-77.036496,2023-02-17,None,0,2023,invierno,2,4,1,1,11.5,3,65
6,20230218,cloudy,3,12,11,Washington,38.897588,-77.036496,2023-02-18,None,0,2023,invierno,2,5,0,1,11.5,3,65


In [ ]:
dic_columnas = {'instant': 'registro','dteday': 'fecha',
                 'season': 'estacion', 'yr': 'año',
                 'mnth':'mes', 'holiday':'festivo',
                 'weekday':'dia_semana','workingday':'no_laboral',
                 'weathersit':'clima', 'temp':'temperatura',
                 'atemp':'sens_termica', 'hum':'humedad',
                 'windspeed':'viento'}